# HDBSCAN should work well on embedding representations

In [4]:
import os
import re
import json
import hdbscan

import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from collections import Counter
import matplotlib.pyplot as plt

import lib.helper as helper
import lib.embedding_models as reps

from importlib import reload

%matplotlib inline

## 1.  Retrieve Corpus

The corpus is being scraped by the "run_news_scrapes.py" script (and windows task scheduler) every 12 hours, a bit past midday and a bit past midnight.

The "bing" corpus are news titles and text extracts gotten from the bing news search API, using a few Home Office - related keywords.

The "RSS" corpus is plugged directly into a number of RSS feeds for world news sites and local british news sites, with no filters for news story types or subjects applied.

In [3]:
# Should be same path for all my PC's, it's where each scrape goes as a separate json file.
storage_path = "D:/Dropbox/news_crow/scrape_results"

# "bing" is targeted news search corpus, "RSS" is from specific world and local news feeds.
corpus_type = "RSS"

# Load up
corpus = helper.load_clean_corpus(storage_path, corpus_type)

# Make sure after cleaning etc it's indexed from 0
corpus.reset_index(inplace=True)
corpus.index.name = "node"

# See how it turned out
print(corpus.shape)
corpus.head()

Total files: 325
9.8 percent of files read.
19.7 percent of files read.
29.5 percent of files read.
39.4 percent of files read.
49.2 percent of files read.
59.1 percent of files read.
68.9 percent of files read.
78.8 percent of files read.
88.6 percent of files read.
98.5 percent of files read.
(79873, 9)


,index,title,summary,date,link,source_url,retrieval_timestamp,origin,clean_text
node,,,,,,,,,
0,0,Hurricane Dorian lashes US as Bahamas counts cost,"Life-threatening US storm surges are feared, a...","Thu, 05 Sep 2019 16:03:44 GMT",https://www.bbc.co.uk/news/world-us-canada-495...,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Hurricane Dorian lashes US as Bahamas counts c...
1,1,Kohistan video murders: Three guilty in 'honou...,They are relatives of a group of Pakistani wom...,"Thu, 05 Sep 2019 13:53:17 GMT",https://www.bbc.co.uk/news/world-asia-49592540,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Kohistan video murders: Three guilty in 'honou...
2,2,MH17 Ukraine plane crash: 'Key witness' released,A Ukrainian court releases a potentially key w...,"Thu, 05 Sep 2019 13:46:06 GMT",https://www.bbc.co.uk/news/world-europe-49591148,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,MH17 Ukraine plane crash: 'Key witness' releas...
3,3,Article 370: The weddings 'ruined' by Kashmir'...,Indian-administered Kashmir is under a securit...,"Thu, 05 Sep 2019 07:32:34 GMT",https://www.bbc.co.uk/news/world-asia-india-49...,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Article 70: The weddings 'ruined' by Kashmir's...
4,4,Syria war: Turkey warns Europe of new migrant ...,President Erdogan demands international help t...,"Thu, 05 Sep 2019 16:11:48 GMT",https://www.bbc.co.uk/news/world-europe-49599297,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Syria war: Turkey warns Europe of new migrant ...


## 2.  Build Text Model (Representation, eg; word2vec, entities list...)

- Trying with the world corpus and with the bing corpus, neither worked with InferSent.  Suspect the problem lies in the PCA step, which may not be working well on this high-dimensional (vector length = 4096) form.
- Summed keywords works rather better with the world corpus.
- Summed keywords still fail the bing/home office corpus, giving me a cluster about "immigration" and a cluster for the American Supreme Court.

In [ ]:
# Windows didn't play nicely with the vector datasets, Some obscure encoding problem (python in Conda
# kept trying to decode using cp1252 regardless of whatever other options I specified!)
# Solution; rewrite file and drop any characters the Windows encoder refuses to recognise.
# I shouldn't loose too much info.
with open('./lib/InferSent/dataset/fastText/crawl-300d-2M.vec', "r", encoding="cp1252", errors="ignore") as infile:
    with open('./lib/InferSent/dataset/fastText/crawl-300d-2M_win.vec', "wb") as outfile:
        for line in infile:
            outfile.write(line.encode('cp1252'))

In [ ]:
infersent = reps.InferSentModel(list(corpus['clean_text']),
                                list(corpus['clean_text']),
                                W2V_PATH = './lib/InferSent/dataset/fastText/crawl-300d-2M_win.vec')

embeddings = infersent.get_embeddings()

In [ ]:
reload(reps)

In [ ]:
# Whereas this worked first time!
glove = reps.NounGloveWordModel(list(corpus['clean_text']), list(corpus['clean_text']))

embeddings = glove.get_embeddings()

In [ ]:
# Turn that into a DF for me
embeddings_df = pd.DataFrame({"clean_text": list(embeddings.keys()),
                              "embeddings": list(embeddings.values())})
embeddings_df.shape

In [ ]:
embeddings_df.head()

## 2a.  Try a really simple averaged word vector model!

With a complex noun extraction function 'cause that part's slow so I multi-threaded it.

In [85]:
from gensim.models import Word2Vec
import spacy
nlp = spacy.load('en_core_web_sm')
from gensim.models.phrases import Phrases, Phraser

In [86]:
def get_phrased_nouns(sentences):
    """ Use spacy to get all of the actual entities, conjoin bigram nouns. """

    # Get the lists of nouns
    noun_lists = []
    for doc in sentences:
        parsed = nlp(doc)
        noun_lists.append([token.lemma_ for token in parsed if token.pos_ == 'PROPN'])

    # Build the phrase model
    phrases = Phrases(noun_lists, min_count=5, threshold=0.5)

    # Get the set of phrases present in the model
    results = []
    for nouns in noun_lists:
        results.append(phrases[nouns])

    return results

# Get phrase-conjoined, lemmatized tokens
test = get_phrased_nouns(corpus['clean_text'])

# Detect and conjoin bigrams
model = Word2Vec(test, size=100, window=5, min_count=1, workers=10)

In [87]:
def get_averaged_vec(token_list, model):
    
    vecs = []
    for token in token_list:
        try:
            vector = model.wv[token]
        except: 
            vector = np.zeros(100)
        vecs.append(vector)
    
    if len(vecs) > 0:
        return np.mean(np.asarray(vecs), axis=0)
    else:
        return np.zeros(100)

In [88]:
vectors = [get_averaged_vec(tokens, model) for tokens in test]

In [89]:
vectors[0:2]

[array([ 1.37630701e-01,  2.49609962e-01, -2.17296302e-01,  1.54974714e-01,
        -7.05686882e-02,  1.02861106e-01, -7.65072405e-02,  1.12681039e-01,
        -1.21565104e-01, -7.06575289e-02, -1.18451290e-01,  9.48330387e-02,
        -2.10751910e-02, -1.42990738e-01,  4.45829332e-02,  3.61130544e-04,
        -1.05684064e-01, -2.65449509e-02,  1.66359499e-01,  1.65221468e-01,
        -4.87100966e-02, -2.78740134e-02, -1.90780833e-01, -6.35896390e-03,
        -3.12371310e-02, -7.98278153e-02, -8.10247362e-02, -7.13581815e-02,
         4.88692038e-02, -5.04988991e-02,  1.45506963e-01, -4.72934358e-02,
        -4.50976305e-02,  1.66130096e-01,  9.92662832e-03,  3.45734246e-02,
         2.89818853e-01, -1.65920645e-01, -1.32138118e-01, -4.05131698e-01,
        -1.89975966e-02,  5.08026071e-02,  6.34343848e-02, -1.81393862e-01,
         1.05653495e-01, -1.05092851e-02, -4.65605222e-02, -1.76114123e-03,
         5.56282997e-02,  1.86172931e-03,  1.13084994e-01,  1.45539880e-01,
         4.5

## 3. Cluster Text

This is the part where the pipelines get a little more experimental

In [90]:
embeddings_array = np.vstack(vectors)
#embeddings_array = np.vstack(embeddings_df['embeddings'])

In [91]:
embeddings_array.shape

(79873, 100)

In [92]:
# First, PCA the data
pca = PCA(n_components=20, svd_solver='full')

embeddings_pca = pca.fit_transform(embeddings_array)

In [93]:
embeddings_pca.shape

(79873, 20)

In [94]:
print(pca.explained_variance_ratio_)
print(pca.singular_values_) 

[8.71977388e-01 6.50198608e-02 4.16280482e-02 7.13385746e-03
 4.84881105e-03 3.45842719e-03 2.06900392e-03 1.95709145e-03
 8.72188947e-04 4.69995133e-04 1.38385991e-04 7.84278803e-05
 6.89012746e-05 3.37457910e-05 2.69592932e-05 2.35481953e-05
 2.08282995e-05 1.57363132e-05 1.42014823e-05 1.00891502e-05]
[533.65907419 145.72508076 116.60154679  48.26957786  39.79503742
  33.60861422  25.99512703  25.28231549  16.87782877  12.38961945
   6.72291084   5.06111873   4.7437841    3.31987012   2.96732789
   2.7732569    2.6081838    2.26705879   2.15366487   1.81525892]


In [95]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=5)
clusterer.fit(embeddings_pca)
pd.unique(clusterer.labels_)

array([  -1,  675, 1248, ...,  142,  427, 1209], dtype=int64)

In [96]:
len(pd.unique(clusterer.labels_))

1375

In [97]:
Counter(clusterer.labels_)

Counter({-1: 56243,
         675: 72,
         1248: 7,
         1337: 4273,
         242: 20,
         1144: 19,
         376: 12,
         385: 17,
         687: 7,
         969: 17,
         151: 6,
         482: 11,
         463: 5,
         425: 23,
         1094: 6,
         68: 6,
         1258: 142,
         30: 81,
         362: 20,
         1052: 16,
         795: 6,
         24: 25,
         862: 6,
         1037: 8,
         153: 33,
         96: 12,
         691: 8,
         447: 26,
         16: 33,
         54: 14,
         235: 12,
         573: 51,
         197: 42,
         92: 10,
         11: 62,
         609: 18,
         288: 112,
         1359: 55,
         648: 15,
         1160: 24,
         1057: 16,
         809: 5,
         0: 20,
         125: 10,
         448: 8,
         676: 10,
         20: 105,
         508: 27,
         788: 9,
         1355: 15,
         349: 12,
         481: 9,
         361: 12,
         328: 39,
         917: 38,
         297: 100